In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from transformer import LlamaLMHeadModel
from transformer import Config

config = Config()
llm = LlamaLMHeadModel(config)
llm = llm.to(config.device)
input_ids = torch.randint(0, config.vocab_size, (32, 128), device=config.device)
loss, logits = llm(input_ids, input_ids.clone())
print(f'Loss: {loss.item()}')


Loss: 673.03564453125


In [ ]:
# The core training begins from here.

import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.load("nanollama_tokenizer.model")
text = "The awful thing is that beauty is mysterious as well as terrible. God and the devil are fighting there and the battlefield is the heart of man. - Fyodor Dostoevsky, The Brothers Karamazov"
ids = sp.encode(text)
tokens = sp.encode(text, out_type=str)
print(ids)
print(tokens)

[75, 14926, 721, 8206, 96, 92, 10799, 96, 8591, 11, 411, 72, 696, 72, 883, 399, 41, 15929, 2701, 33, 8, 2069, 57, 146, 3833, 533, 33, 8, 7177, 96, 8, 5111, 21, 834, 15929, 15906, 1, 157, 15924, 125, 26, 177, 212, 15912, 2961, 11765, 15927, 75, 9272, 187, 15913, 8189, 80, 1265, 466]
['▁The', '▁aw', 'ful', '▁thing', '▁is', '▁that', '▁beauty', '▁is', '▁myst', 'er', 'ious', '▁as', '▁well', '▁as', '▁ter', 'rib', 'le', '.', '▁God', '▁and', '▁the', '▁dev', 'il', '▁are', '▁fighting', '▁there', '▁and', '▁the', '▁battlefield', '▁is', '▁the', '▁heart', '▁of', '▁man', '.', '▁', '―', '▁F', 'y', 'od', 'or', '▁D', 'ost', 'o', 'ev', 'sky', ',', '▁The', '▁Bro', 'ther', 's', '▁Kar', 'am', 'az', 'ov']


In [3]:
from typing import Union, List, Optional
import sentencepiece as spm
import torch

class NanoLlamaTokenizer:
    def __init__(self, model_path: str = "nanollama_tokenizer.model",
                 bos_id: int = 2, eos_id: int = 3, pad_id: int = 4):
        self.tokenizer = spm.SentencePieceProcessor()
        self.tokenizer.load(model_path)
        self.bos_id = bos_id
        self.eos_id = eos_id
        self.pad_id = pad_id

    def encode(self, text: Union[str, List[str]], add_bos: bool = True, add_eos: bool = True,
               max_length: Optional[int] = None, pad: bool = True, return_tensor: bool = False
               ) -> Union[List[int], List[List[int]], torch.Tensor]:
        def _encode_single(s: str) -> List[int]:
            ids = self.tokenizer.encode(s)
            if add_bos:
                ids = [self.bos_id] + ids
            if add_eos:
                ids = ids + [self.eos_id]
            if max_length:
                ids = ids[:max_length]
            if pad and max_length:
                ids += [self.pad_id] * (max_length - len(ids))
            return ids

        if isinstance(text, str):
            encoded = _encode_single(text)
        elif isinstance(text, list):
            encoded = [_encode_single(s) for s in text]
        else:
            raise ValueError("Input must be a string or a list of strings.")

        if return_tensor:
            if isinstance(encoded[0], list):
                return torch.tensor(encoded)
            else:
                return torch.tensor([encoded])
        return encoded

    def decode(self, ids: Union[List[int], List[List[int]], torch.Tensor]) -> Union[str, List[str]]:
        if isinstance(ids, torch.Tensor):
            ids = ids.tolist()
        if isinstance(ids[0], list):
            return [self.tokenizer.decode(id_seq) for id_seq in ids]
        else:
            return self.tokenizer.decode(ids)

    def __call__(self, text: Union[str, List[str]], **kwargs):
        return self.encode(text, **kwargs)


In [4]:
from torch.utils.data import Dataset, DataLoader
import torch

class TokenizedTextDataset(Dataset):
    def __init__(self, file_path: str, tokenizer: NanoLlamaTokenizer, block_size: int = 128):
        with open(file_path, 'r', encoding='utf-8') as f:
            self.text = f.read()
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.tokens = self.tokenizer.encode(self.text, add_bos=False, add_eos=False, pad=False)

    def __len__(self):
        return len(self.tokens) // self.block_size

    def __getitem__(self, idx):
        i = idx * self.block_size
        input_ids = self.tokens[i: i + self.block_size]
        target_ids = input_ids[1:] + [self.tokenizer.pad_id]  # Shifted for language modeling
        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "labels": torch.tensor(target_ids, dtype=torch.long),
        }


def collate_fn(batch, pad_id=4):
    input_ids = [item['input_ids'] for item in batch]
    labels = [item['labels'] for item in batch]
    max_len = max(len(x) for x in input_ids)

    input_ids_padded = torch.stack([torch.cat([x, torch.full((max_len - len(x),), pad_id)]) for x in input_ids])
    labels_padded = torch.stack([torch.cat([x, torch.full((max_len - len(x),), -100)]) for x in labels])  # -100 is ignored in loss

    attention_mask = (input_ids_padded != pad_id).long()

    return {
        "input_ids": input_ids_padded,
        "labels": labels_padded,
        "attention_mask": attention_mask
        }

tokenizer = NanoLlamaTokenizer("nanollama_tokenizer.model")
dataset = TokenizedTextDataset("nanollama_training_corpus.txt", tokenizer, block_size=128)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

total_batches = len(dataloader)
print("Total number of batches:", total_batches)
# Get one batch from the dataloader
example_batch = next(iter(dataloader))
# Check shapes of the batch tensors
print("Batch input_ids shape:", example_batch["input_ids"].shape)
print("Batch labels shape:", example_batch["labels"].shape)
print("Batch attention_mask shape:", example_batch["attention_mask"].shape)


Total number of batches: 3064
Batch input_ids shape: torch.Size([8, 128])
Batch labels shape: torch.Size([8, 128])
Batch attention_mask shape: torch.Size([8, 128])


In [8]:
import torch
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
import os

def train(model, dataloader, optimizer, device, epochs=1, save_dir="checkpoints"):
    os.makedirs(save_dir, exist_ok=True)

    model.to(device)
    model.train()

    loss_fn = nn.CrossEntropyLoss()
    scaler = GradScaler()

    global_step = 0

    for epoch in range(epochs):
        total_loss = 0

        for step, batch in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}")):
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()

            with autocast():
                logits = model(input_ids)  # (B, T, vocab_size)
                loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            global_step += 1

            # Log every 10 batches
            if global_step % 10 == 0:
                print(f"Step {global_step} | Loss: {loss.item():.4f}")

            # Save checkpoint every 500 batches
            if global_step % 500 == 0:
                checkpoint_path = os.path.join(save_dir, f"checkpoint_step{global_step}.pt")
                torch.save(model.state_dict(), checkpoint_path)
                print(f"Saved checkpoint at step {global_step} to {checkpoint_path}")

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1} complete | Avg Loss: {avg_loss:.4f}")

        # Save model at end of epoch
        epoch_path = os.path.join(save_dir, f"model_epoch{epoch+1}.pt")
        torch.save(model.state_dict(), epoch_path)
        print(f"Saved model at end of epoch {epoch+1} to {epoch_path}")


In [9]:
model = LlamaLMHeadModel(config)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device in usage: {device}')
tokenizer = NanoLlamaTokenizer("nanollama_tokenizer.model")
dataset = TokenizedTextDataset("nanollama_training_corpus.txt", tokenizer, block_size=128)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr=3e-4,           # Learning rate
                              betas=(0.9, 0.95), # Momentum terms
                              eps=1e-8,          # Epsilon for numerical stability
                              weight_decay=0.1)  # Helps generalization


train(model=model,
      device=device,
      dataloader=dataloader, optimizer=optimizer, epochs=1)

Device in usage: cuda


C:\Users\HP\AppData\Local\Temp\ipykernel_25788\1380132320.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1:   0%|          | 0/3064 [00:00<?, ?it/s]C:\Users\HP\AppData\Local\Temp\ipykernel_25788\1380132320.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1:   0%|          | 10/3064 [00:39<3:21:24,  3.96s/it]

Step 10 | Loss: 108.7156


Epoch 1:   1%|          | 20/3064 [01:19<3:23:23,  4.01s/it]

Step 20 | Loss: 81.2143


...

Step 3050 | Loss: 6.5972


Epoch 1: 100%|█████████▉| 3060/3064 [3:23:28<00:15,  3.86s/it]

Step 3060 | Loss: 6.8759


Epoch 1: 100%|██████████| 3064/3064 [3:23:41<00:00,  3.99s/it]


Epoch 1 complete | Avg Loss: 12.7187
Saved model at end of epoch 1 to checkpoints\model_epoch1.pt


In [10]:
torch.save(model.state_dict(), "nanollama_weights.pth")

In [15]:
import torch

def generate_autoregressive(
    model,
    tokenizer,
    prompt: str,
    max_new_tokens: int = 50,
    temperature: float = 1.0,
    top_k: int = 50,
    eos_token_id: int = 3,
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensor=True).to(device)
    
    with torch.no_grad():
        for _ in range(max_new_tokens):
            outputs = model(input_ids)  # logits: [1, seq_len, vocab_size]
            logits = outputs[:, -1, :] / temperature  # take last token's logits
            if top_k is not None:
                top_k_logits, top_k_indices = torch.topk(logits, top_k)
                probs = torch.nn.functional.softmax(top_k_logits, dim=-1)
                next_token = top_k_indices[0, torch.multinomial(probs, 1)]
            else:
                probs = torch.nn.functional.softmax(logits, dim=-1)
                next_token = torch.multinomial(probs, 1)

            # Squeeze the next_token and concatenate with input_ids
            next_token = next_token.squeeze(1)  # Remove extra dimension
            input_ids = torch.cat([input_ids, next_token.unsqueeze(1)], dim=1)  # Concatenate

            if next_token.item() == eos_token_id:
                break

    return tokenizer.decode(input_ids[0].tolist())


In [16]:
generated = generate_autoregressive(
    model=model,
    tokenizer=tokenizer,
    prompt="The future of artificial intelligence is",
    max_new_tokens=100,
    top_k=40
)

print(generated)


<s> The future of artificial intelligence is</s> as the S-Du-croy-nas-Ata in the Middle East" of many Roman states, a matter of the natural result in the early as a third-in-Ap-Doni or Staci of a result of a century, the Sayian, the Bot, or others, the first century, social structure or more visible matter of some to the species of a star, a very similar of the social social structure of the general
